In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import re


In [2]:
data = pd.read_csv(r'C:\Users\AinXD\Documents\dataset\judul abstrak.csv')
judul = data['Judul']
abstrak = data['Abstract']
print (data)

                                               Judul  \
0  SISTEM PENDUKUNG KEPUTUSAN UNTUK MENENTUKAN JU...   
1  Sistem Pendukung Keputusan Pemilihan Topik Skr...   
2  Sistem Pendukung Keputusan Rekomendasi Judul S...   
3  Sistem Penunjang Keputusan Metode Technique Or...   
4  SISTEM PENDUKUNG KEPUTUSAN MENENTUKAN JUDUL SK...   

                                            Abstract  
0  Dalam Perguruan Tinggi, kelulusan mahasiswa bi...  
1  Jurusan Teknik Informatika di STMIK Pelita Nus...  
2  Penerapan metode ELECTRE dalam sistem pendukun...  
3  Informasi menjadi salah satu kebutuhan dalam s...  
4  Proses pendidikan yang harus ditempuh untuk me...  


In [3]:
# pre procesing

def preprocess_text(text):
    if isinstance(text, str):
        
# Menghapus angka
        text = re.sub(r'\d+', '', text)  
    
# Menghapus karakter khusus
        text = re.sub(r'[^\w\s]', '', text)
    
# Mengubah teks menjadi lowercase
        text = text.lower() 
    return text

In [4]:
for key in data:
    data[key] = data[key].apply(preprocess_text)


In [5]:
# pre processing
def preprocess_text(text):
    stop_words = set(stopwords.words('indonesian'))
    words = word_tokenize(str(text).lower())
    words = [word for word in words if word.isalnum() and word not in stop_words]
    return words

In [6]:
# Membangun graf berbasis kata-kunci
def build_keyword_graph(texts):
    sentences = [sent_tokenize(text) for text in texts]
    words = [preprocess_text(text) for text in texts]
    unique_words = list(set([word for sublist in words for word in sublist]))

    word_index = {word: index for index, word in enumerate(unique_words)}
    graph = np.zeros((len(unique_words), len(unique_words)))

    for sentence in words:
        for i in range(len(sentence)):
            for j in range(i+1, len(sentence)):
                word_i = sentence[i]
                word_j = sentence[j]
                graph[word_index[word_i]][word_index[word_j]] += 1
                graph[word_index[word_j]][word_index[word_i]] += 1

    return graph, unique_words

In [7]:
def extract_keywords(texts, top_n=3):
    graph, unique_words = build_keyword_graph(texts)
    similarity_matrix = cosine_similarity(graph)

    scores = np.sum(similarity_matrix, axis=1)
    top_indices = np.argsort(scores)[-top_n:][::-1]
    keywords = [unique_words[index] for index in top_indices]

    # Menggabungkan dua kata pertama dalam satu kata kunci
    combined_keywords = [" ".join(keyword.split()[:2]) for keyword in keywords]

    return combined_keywords

In [10]:
# Ekstraksi kata-kunci dari data judul dan abstrak
for i in range(len(judul)):
    title = judul[i]
    abstract = abstrak[i]
    text = str(abstract) + " " + str(title)
    keywords = extract_keywords([text], top_n=3)

    print("Abstrak:", abstract)
    print("Kata-kunci:", keywords)
    print("--------------------")

Abstrak: dalam perguruan tinggi kelulusan mahasiswa bisa didapatkan jika sudah mengerjakan skripsi bagi mahasiswa strata s skripsi merupakan tugas akhir yang harus dikerjakan oleh mahasiswa sesuai bidang keilmuannya kewajiban untuk mengerjakan skripsi yang sudah didasari ilmu â ilmu pada saat kuliah ternyata masih membuat mahasiswa bingung dalam menentukan judul skripsi penentuan judul skripsi menjadi sangat penting jika mahasiswa akan mengerjakan skripsi pada penelitian ini diangkat kasus demikian dengan memberikan bantuan kepada mahasiswa untuk menentukan judul skripsi melalui sebuah perhitungan metode yang ada pada sistem pendukung keputusan untuk menentukan judul skripsi menggunakan metode topsis dalam perhitungan dengan menggunakan kriterianya yaitu nilai akademik histories project keaktifan penelitian pelatihan minat topsis dipilih karena merupakan metode dengan konsep sederhana dan mudah dipahami serta praktis metode ini memiliki hasil jarak terkecil dari solusi ideal positif da